## 0. Importing libraries

In [43]:
from stackapi import StackAPI
from pandas import datetime
import pandas as pd
from functools import reduce
from sklearn.utils import shuffle

## 1. Custom functions created

In [44]:
def create_dataframe(data, name):
    
    '''
    Function that creates a CSV file from the dictionary data object returned by the API.
    
    Input 
           : data -> data from API -> type : dictionary
           : name -> name of the CSV file -> type : string
    
    Output : Returns nothing
    '''
    
    data_items = data["items"]
    data_frame = pd.DataFrame()
    question_list = []
    tags_list = []

    for i in data_items:
        question_list.append(i["title"])
        tags_list.append("".join(["__label__"+str(i) + " " for i in i["tags"]]))
    
    data_frame["Question"] = question_list
    data_frame["Labels"] = tags_list
    data_frame.to_csv(name + ".csv", header = True)

## 2. Data Generation

### 2.1 Setting parameters for API call

In [8]:
# For Authentication (OAuth call)
c_key = "qiEDB0WIq3e3MM7LRZB*3Q(("
c_secret = "L7Pv1pOb2VJxDvBy9qH38A((" 

# Configuring the APIs
## INTERNET SECURITY
SITE_IS = StackAPI('security', key = c_key, client_secret = c_secret)

## STATISTICS
SITE_ST = StackAPI('stats', key = c_key, client_secret = c_secret)

## PHYSICS
SITE_PH = StackAPI('physics', key = c_key, client_secret = c_secret)

# TOTAL QUESTION = page_sizes * max_pages = 50,000
## INTERNET SECURITY
SITE_IS.page_sizes = 100
SITE_IS.max_pages = 500

## STATISTICS
SITE_ST.page_sizes = 100
SITE_ST.max_pages = 500

## PHYSICS
SITE_PH.page_sizes = 100
SITE_PH.max_pages = 500

### 2.2 Calling the API

In [4]:
data_IS = SITE_IS.fetch("questions")
data_ST = SITE_ST.fetch("questions")
data_PH = SITE_PH.fetch("questions")

In [12]:
create_dataframe(data_IS, "IS")
create_dataframe(data_ST, "STATS")
create_dataframe(data_PH, "PHYSICS")

### 2.3 Converting data to appropriate manner

In [45]:
df_IS = pd.read_csv("IS.csv", index_col = 0)
df_ST = pd.read_csv("STATS.csv", index_col = 0)
df_PH = pd.read_csv("PHYSICS.csv", index_col = 0)

In [46]:
df_list = [df_IS, df_PH, df_ST]
df_final = pd.concat(df_list)

In [47]:
df_final = shuffle(df_final)
df_final.reset_index(drop = True, inplace = True)

In [48]:
df_final.to_csv("final.csv", header = True)

In [50]:
string = ""
with open("final_data.txt", 'w', encoding = 'utf-8') as file:
    for index, row in df_final.iterrows():
        string += (row["Labels"] + row["Question"])
        string += '\n'
    file.write(string)